In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    access_token = userdata.get('DEFORMER_TOKEN')
    !pip install git+https://$access_token@github.com/ay94/deformer-extractor.git@error-handling

In [6]:
from experiment_utils import colab
from experiment_utils.general_utils import FileHandler
from experiment_utils.pipelines import OutputGenerationPipeline, AnalysisExtractionPipeline
from experiment_utils.evaluation import Metrics
from experiment_utils.train import DatasetManager
from experiment_utils.configurations import  ExtractionConfigManager, ResultsConfigManager
from experiment_utils.analysis import AnalysisWorkflowManager, UMAPTransformer, LabelAligner

In [8]:
base_folder = colab.init('My Drive')



2024-08-09 19:34:44 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/scripts/My Drive
2024-08-09 19:34:44 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [140]:
finetuning_path = base_folder / 'Final Year Experiments/Class Imbalance/1_fineTuning'
fh = FileHandler(finetuning_path)


import torch
model_name='arabertv02'
data_name='ANERCorp_CamelLab'
training_outputs = fh.load_pickle(
            f"evalOutputs/{model_name}_{data_name}_regular_outputs.pkl"
        )

load_model_path = fh.file_path / f"trainOutputs/{model_name}_{data_name}_regular.bin"
model = torch.load(load_model_path, map_location=torch.device('cpu'))

Mounted at /content/drive


In [4]:
# saved durinig training
results_dict = {
            "token_results": training_outputs.test_metrics.skl_results,
            "token_report": training_outputs.test_metrics.skl_report,
            "token_outputs": training_outputs.test_metrics.skl_output,
            "entity_results": training_outputs.test_metrics.seq_results,
            "entity_report": training_outputs.test_metrics.seq_report,
            "entity_outputs": training_outputs.test_metrics.seq_output
        }

evaluation_results = Metrics.from_dict(results_dict)

In [9]:
config_path = base_folder / 'Final Year Experiments/Thesis-Experiments/scripts'
corpora_path = base_folder / 'Final Year Experiments/Thesis-Experiments/ExperimentData'
config_manager = ExtractionConfigManager(config_path, 'config.yaml')

2024-08-09 19:34:59 - ERROR - YAML file not found: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/scripts/config.yaml


In [6]:
data_manager = DatasetManager(corpora_path, config_manager.dataset_name, config_manager.tokenization_config)

INFO:root:Tokenization Config validated successfully


# final results

In [7]:
output_generation_pipeline = OutputGenerationPipeline(model, data_manager, config_manager, config_manager.model_path)

In [8]:
output_generation_pipeline.run('test')

INFO:root:Loading pretrained model from: aubmindlab/bert-base-arabertv02
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Generating model outputs for split: test
INFO:root:Training Config validated successfully
INFO:root:Specific Split test being processed
INFO:root:Loading Preprocesso

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

...

  0%|          | 0/4149 [00:00<?, ?it/s]

INFO:root:Extracting train subwords


  0%|          | 0/4149 [00:00<?, ?it/s]

INFO:root:Processing test split


  0%|          | 0/961 [00:00<?, ?it/s]

In [9]:
analysis_extraction_pipeline = AnalysisExtractionPipeline(
    output_pipeline=output_generation_pipeline,
    evaluation_results=evaluation_results,  # Assuming 'evaluation_results' is a variable holding model evaluation metrics
    config_manager=config_manager,
    split='test'
)


In [10]:
analysis_extraction_pipeline.run()

INFO:root:UMAP Config validated successfully
INFO:root:Analysis extraction pipeline initialized successfully.
INFO:root:Extracting model features...
INFO:root:Extracting tokenization features...
INFO:root:Aligning labels...
INFO:root:Applying UMAP...
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


UMAP(angular_rp_forest=True, metric='cosine', n_jobs=1, random_state=1, verbose=True)
Thu Aug  8 20:00:45 2024 Construct fuzzy simplicial set
Thu Aug  8 20:00:45 2024 Finding Nearest Neighbors
Thu Aug  8 20:00:45 2024 Building RP forest with 14 trees
Thu Aug  8 20:00:52 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	 6  /  15
	Stopping threshold met -- exiting after 6 iterations
Thu Aug  8 20:01:14 2024 Finished Nearest Neighbor Search
Thu Aug  8 20:01:17 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Thu Aug  8 20:01:41 2024 Finished embedding


INFO:root:Extracting model features...
INFO:root:Extracting tokenization features...
INFO:root:Applying UMAP...
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

Extract Subwords:   0%|          | 0/23049 [00:00<?, ?it/s]

Calculate Consistency:   0%|          | 0/29711 [00:00<?, ?it/s]

INFO:root:Annotating token entropy...
INFO:root:Annotating word entropy...
INFO:root:Annotating entity...
INFO:root:Annotating error types...
INFO:root:Annotating prediction entropy...
INFO:root:Annotating pretrained coordinates...
INFO:root:Analysis workflow execution time: 134.376362323761 seconds
INFO:root:Computing attention similarities


Computing attention similarities:   0%|          | 0/961 [00:00<?, ?it/s]

INFO:root:Comparing weights


In [13]:
analysis_extraction_pipeline.analysis_data

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
0,0,0,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS]@#0@#0,[CLS],True,...,0.000042,2.961268e-06,0.000039,0.000007,0.002194,3.169925,0.999869,0.314224,16.240751,8.657773
1,0,1,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,...,0.736495,1.161346e-03,0.001891,0.000154,0.987601,3.169925,0.736495,0.232984,12.008531,3.497766
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,0.978258,7.750425e-03,0.000166,0.000185,0.195380,3.169925,0.978258,0.306596,12.091125,1.360525
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,0.000321,5.764136e-05,0.000125,0.000077,0.014571,3.169925,0.998964,0.313903,8.703134,5.905618
4,0,4,غيث,غيث,[غيث],غيث,B-PERS,غيث@#4@#0,B-PERS,True,...,0.000537,3.313661e-04,0.000873,0.000214,0.192781,3.169925,0.977510,0.306351,12.379341,1.654007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29706,960,10,للوليد,##ليد,"[للو, ##ليد]",IGNORED,IGNORED,IGNORED@#10@#960,IGNORED,True,...,0.000287,1.335926e-04,0.000204,0.000044,0.056429,3.169925,0.994974,0.312494,6.676324,-5.634513
29707,960,11,بن,بن,[بن],بن,I-PERS,بن@#11@#960,I-PERS,True,...,0.000063,1.495592e-04,0.000086,0.000138,0.063753,3.169925,0.994191,0.312217,6.765226,-5.459598
29708,960,12,طلال,طلال,[طلال],طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,0.000067,1.289662e-04,0.000155,0.000110,0.038872,3.169925,0.996783,0.313133,6.672010,-5.624885
29709,960,13,.,.,[.],.,O,.@#13@#960,O,True,...,0.000005,6.299709e-07,0.000013,0.000004,0.000704,3.169925,0.999963,0.314256,10.402867,9.328933


In [17]:
analysis_extraction_pipeline.centroids_avg_similarity_matrix

,NER_Label,Centroid_0,Centroid_1,Centroid_2,Centroid_3,Centroid_4,Centroid_5,Centroid_6,Centroid_7,Centroid_8
0,B-LOC,0.078337,0.065467,0.152045,0.039408,0.770973,0.305903,0.181040,0.106156,0.053772
1,B-MISC,0.282262,0.277508,0.182359,0.090346,0.140864,0.444202,0.155182,0.331131,0.267634
2,B-ORG,0.188452,0.177411,0.268440,0.091939,0.267911,0.630651,0.204750,0.231388,0.161305
3,B-PERS,0.226804,0.182863,0.784925,0.395777,0.154057,0.302737,0.131362,0.194788,0.136956
4,I-LOC,0.030668,-0.016206,0.081839,0.206061,0.326221,0.099766,0.501919,-0.011610,-0.120578
5,I-MISC,0.282356,0.264430,0.096229,0.173163,0.118191,0.195529,0.393991,0.279202,0.208820
6,I-ORG,0.153003,0.110695,0.133179,0.285359,0.125994,0.266057,0.629765,0.116400,0.005522
7,I-PERS,0.135733,0.073544,0.429828,0.830005,0.050107,0.123359,0.349911,0.060859,-0.047083
8,O,0.867645,0.875960,0.209171,0.061224,0.067810,0.264525,0.104607,0.862579,0.799744


In [ ]:
self.outputs =  {
                  "analysis_data": analysis_data,
                  "entity_report": self.evaluation_results.entity_report,
                  "entity_results": self.evaluation_results.entity_results,
                  "token_report": self.evaluation_results.token_report,
                  "token_results": self.evaluation_results.token_results,
                  "average_silhouette_score": average_silhouette_score,
                  "kmeans_metrics": kmeans_metrics,
                  "centroids_avg_similarity_matrix": centroids_avg_similarity_matrix,
                  "attention_similarity_matrix": attention_similarity_matrix,
                  "attention_weights_similarity": attention_weights_similarity,
                  "entity_confusion_data": entity_confusion_data,
            }

In [12]:
analysis_extraction_pipeline.analysis_data.columns

Index(['sentence_ids', 'token_positions', 'words', 'tokens', 'word_pieces',
       'core_tokens', 'true_labels', 'token_selector_id', 'pred_labels',
       'agreements', 'x', 'y', 'labels', 'losses', 'token_ids', 'global_id',
       'true_token_score', 'pred_token_score', 'k=3', 'boundary_clusters',
       'k=4', 'entity_clusters', 'k=9', 'token_clusters', 'consistency_count',
       'inconsistency_count', 'total_train_occurrences', 'local_token_entropy',
       'token_max_entropy', 'dataset_token_entropy', 'local_word_entropy',
       'word_max_entropy', 'dataset_word_entropy', 'tokenization_rate',
       'tr_entity', 'pr_entity', 'error_type', 'O', 'B-PERS', 'I-PERS',
       'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC',
       'prediction_entropy', 'prediction_max_entropy', 'confidence',
       'variability', 'pre_x', 'pre_y'],
      dtype='object')

In [14]:
from dataclasses import dataclass, field
from typing import Dict, Any, Optional
from pathlib import Path
@dataclass
class ResultsSavingConfig:
    output_root: str
    analysis_data: Dict[str, Dict[str, str]] = field(default_factory=dict)

    def __post_init__(self):
        # Validate the structure of the analysis_data dictionary
        for key, value in self.analysis_data.items():
            if not isinstance(value, dict):
                raise ValueError(f"Each entry in analysis_data must be a dictionary, but got {type(value)} for key {key}")
            if 'folder' not in value or 'filename' not in value or 'format' not in value:
                raise ValueError(f"Each entry in analysis_data must contain 'folder', 'filename', and 'format'. Missing in key {key}")



In [35]:
experiment_base_folder = base_folder / 'Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment'

In [36]:
from experiment_utils.configurations import TrainingConfig, ModelConfig, EvaluationConfig, TokenizationConfig, UMAPConfig, ClusteringConfig
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Any
import yaml

@dataclass
class ExtractionConfigManager:
    def __init__(self, config_path: Path, file_name: str):
        config_fh = FileHandler(config_path)
        self.config = config_fh.load_yaml(file_name)

    @property
    def dataset_name(self) -> str:
        return self.config.get('dataset_name', None)

    @property
    def model_path(self) -> str:
        return self.config.get('model_path', None)

    @property
    def training_config(self) -> TrainingConfig:
        return TrainingConfig.from_dict(self.config.get('training', {}).get('args', {}))

    @property
    def model_config(self) -> ModelConfig:
        return ModelConfig.from_dict(self.config.get('training', {}).get('model', {}))

    @property
    def evaluation_config(self) -> EvaluationConfig:
        return EvaluationConfig.from_dict(self.config.get('training', {}).get('evaluation', {}))

    @property
    def tokenization_config(self) -> TokenizationConfig:
        return TokenizationConfig.from_dict(self.config.get('extraction', {}).get('tokenization', {}))

    @property
    def umap_config(self) -> UMAPConfig:
        return UMAPConfig.from_dict(self.config.get('extraction', {}).get('umap', {}))

    @property
    def clustering_config(self) -> ClusteringConfig:
        return ClusteringConfig.from_dict(self.config.get('extraction', {}).get('clustering', {}))



class ResultsConfigManager:
    def __init__(self, config_path: Path, file_name: str):
        config_fh = FileHandler(config_path)
        self.config = config_fh.load_yaml(file_name)

    @property
    def dataset_name(self) -> Path:
        return self.config.get('dataset_name', None)

    @property
    def model_name(self) -> Path:
        return self.config.get('model_name', None)

    @property
    def analysis_data(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('analysis_data', {})

    @property
    def train_data(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('train_data', {})

    @property
    def entity_report(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('entity_report', {})

    @property
    def token_report(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('token_report', {})

    @property
    def kmeans_results(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('kMeans_results', {})

    @property
    def entity_confusion_data(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('entity_confusion_data', {})

    @property
    def attention_weights_similarity(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('attention_weights_similarity', {})

    @property
    def centroids_avg_similarity_matrix(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('centroids_avg_similarity_matrix', {})

    @property
    def attention_similarity_matrix(self) -> Dict[str, Any]:
        return self.config.get('extraction_data', {}).get('attention_similarity_matrix', {})


In [37]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Any
import yaml
import logging
@dataclass
class ExperimentConfig:
    base_folder: Path
    corpora_path: Path
    extraction_config_file: str
    results_config_file: str
    experiment_name: str
    datasets_and_models: Dict[str, Dict[str, Any]]
    extraction_base: Path = None
    finetuning_base: Path = None

    extraction_manager: 'ExtractionConfigManager' = field(init=False, default=None)
    results_manager: 'ResultsConfigManager' = field(init=False, default=None)

    def __post_init__(self):
        # Make sure paths are correctly initialized
        extraction_config_path = self.base_folder / 'configs'
        results_config_path = self.base_folder / 'configs'
        print(self.extraction_config_file)

        # Initialize managers with the correct paths
        self.extraction_manager = ExtractionConfigManager(extraction_config_path, self.extraction_config_file)
        self.results_manager = ResultsConfigManager(results_config_path, self.results_config_file)


    @classmethod
    def from_yaml(cls, base_folder: Path) -> 'ExperimentConfig':
        """ Initialize from a YAML file """
        config_fh = FileHandler(base_folder / 'configs')
        config_dict = config_fh.load_yaml('experiment_config.yaml')
        print(config_dict)
        return cls.from_dict(config_dict, base_folder)

    @classmethod
    def from_dict(cls, config_dict: Dict[str, Any], base_folder: Path) -> 'ExperimentConfig':
        """ Initialize from a dictionary """
        corpora_path = config_dict.get('corpora_path')
        if corpora_path is None:
            raise ValueError("The 'corpora_path' key is missing from the configura tion.")
        return cls(
            base_folder=base_folder,
            corpora_path=base_folder.parents[1] / config_dict['corpora_path'],
            experiment_name=config_dict['experiment_name'],
            datasets_and_models=config_dict['datasets_and_models'],
            extraction_config_file=config_dict.get('extraction_config_file'),
            results_config_file=config_dict.get('results_config_file'),
        )

    def create_folder_structure(self):
        """ Create the necessary folder structure for the experiment using pathlib """

        # Fine-tuning folders
        self.fine_tuning_base = self.base_folder / 'fine_tuning'
        self.fine_tuning_base.mkdir(parents=True, exist_ok=True)
        for dataset, models in self.datasets_and_models.items():
          for model in models.get('models'):
            model_dir = self.fine_tuning_base / dataset / model
            (model_dir / 'model_files').mkdir(parents=True, exist_ok=True)
            # Fine tuning related outputs

        # Extraction folders
        self.extraction_base = self.base_folder / 'extraction'
        self.extraction_base.mkdir(parents=True, exist_ok=True)
        for dataset, models in self.datasets_and_models.items():
            dataset_dir = self.extraction_base / dataset
            dataset_dir.mkdir(parents=True, exist_ok=True)

            for model in models.get('models'):
                model_dir = dataset_dir / model
                (model_dir / 'analysis').mkdir(parents=True, exist_ok=True)
                (model_dir / 'results').mkdir(parents=True, exist_ok=True)
                (model_dir / 'matrices').mkdir(parents=True, exist_ok=True)

        logging.info(f"Folder structure created under {self.base_folder}")



In [38]:
experiment_base_folder

PosixPath('/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment')

In [39]:
experiment_config = ExperimentConfig.from_yaml(experiment_base_folder)

{'corpora_path': 'ExperimentData', 'datasets_and_models': {'ANERCorp_CamelLab': {'arabertv02': ['extraction_config.yaml', 'results_config.yaml']}, 'dataset_2': {'model_2': ['extraction_config.yaml', 'results_config.yaml']}}}


KeyError: 'experiment_name'

In [40]:
from pathlib import Path
import yaml

class Experiment:
    def __init__(self, config_path):
        self.config = self.from_yaml(config_path)
 
        

        self.corpora_path = Path(self.config['corpora_path'])
        self.datasets_and_models = self.config['datasets_and_models']
    
    
    def from_yaml(self, config_path: Path) -> 'ExperimentConfig':
        """ Initialize from a YAML file """
        config_fh = FileHandler(config_path)
        config_dict = config_fh.load_yaml('experiment_config.yaml')
        return config_dict

    def create_folder_structure(self):
        # Create the base directory
        self.base_path.mkdir(parents=True, exist_ok=True)

        # Iterate through datasets and models to create their respective directories
        for dataset, models in self.datasets_and_models.items():
            for model, files in models.items():
                model_path = self.base_path / dataset / model
                model_path.mkdir(parents=True, exist_ok=True)
                
                # Optionally copy the specified config files to the structure
                extraction_config_path = model_path / files['extraction_config_file']
                results_config_path = model_path / files['results_config_file']
                
                # Initialize or copy existing configs (this could be more advanced if needed)
                self.initialize_config_file(extraction_config_path)
                self.initialize_config_file(results_config_path)
    
    def initialize_config_file(self, config_file_path):
        # This method ensures the config file exists or initializes it with a base structure
        if not config_file_path.exists():
            with config_file_path.open('w') as file:
                yaml.dump({}, file)  # Create an empty or base config

    def load_extraction_config(self, dataset, model):
        # Load the extraction config for a given dataset-model pair
        model_info = self.datasets_and_models[dataset][model]
        config_file = model_info['extraction_config_file']
        config_path = self.base_path / dataset / model / config_file
        
        with config_path.open('r') as file:
            return yaml.safe_load(file)

    def load_results_config(self, dataset, model):
        # Load the results config for a given dataset-model pair
        model_info = self.datasets_and_models[dataset][model]
        config_file = model_info['results_config_file']
        config_path = self.base_path / dataset / model / config_file
        
        with config_path.open('r') as file:
            return yaml.safe_load(file)

    def run_experiment(self):
        # Iterate through datasets and models to run the experiment workflow
        for dataset, models in self.datasets_and_models.items():
            for model in models.keys():
                print(f"Running extraction for dataset: {dataset}, model: {model}")
                extraction_config = self.load_extraction_config(dataset, model)
                
                # Here, you'd insert the logic to perform extraction based on the loaded config
                self.perform_extraction(extraction_config)
                
                print(f"Saving results for dataset: {dataset}, model: {model}")
                results_config = self.load_results_config(dataset, model)
                
                # Logic for saving the results based on the loaded config
                self.save_results(results_config)
    
    def perform_extraction(self, config):
        # Placeholder for the actual extraction logic
        print(f"Performing extraction with config: {config}")
        # Your extraction code goes here

    def save_results(self, config):
        # Placeholder for the actual results saving logic
        print(f"Saving results with config: {config}")
        # Your saving code goes here




In [41]:
# Example usage
experiment = Experiment(experiment_base_folder)


2024-08-09 19:50:03 - ERROR - YAML file not found: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/experiment_config.yaml


TypeError: 'NoneType' object is not subscriptable

In [212]:
experiment.create_folder_structure()
experiment.run_experiment()


['json', 'vaex']

In [210]:
experiment_config.results_manager.config

{'dataset_name': 'ANERCorp_CamelLab',
 'model_name': 'arabertv02',
 'extraction_data': {'analysis_data': {'folder': 'analysis',
   'filename': 'analysis_data',
   'format': ['json', 'vaex']},
  'train_data': {'folder': 'analysis',
   'filename': 'train_data',
   'format': ['json', 'vaex']},
  'entity_report': {'folder': 'results',
   'filename': 'entity_report',
   'format': 'json'},
  'token_report': {'folder': 'results',
   'filename': 'token_report',
   'format': 'json'},
  'kMeans_results': {'folder': 'results',
   'filename': 'kMeans_results',
   'format': 'json'},
  'entity_confusion_data': {'folder': 'results',
   'filename': 'entity_confusion_data',
   'format': 'json'},
  'attention_weights_similarity': {'folder': 'matrices',
   'filename': 'attention_weights_similarity',
   'format': 'json'},
  'centroids_avg_similarity_matrix': {'folder': 'matrices',
   'filename': 'centroids_avg_similarity_matrix',
   'format': 'json'},
  'attention_similarity_matrix': {'folder': 'matrices'

In [225]:
import json
import vaex
from pathlib import Path
from typing import Dict, Any
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

class ResultsSavingPipline:
    def __init__(self, results_config: Dict[str, Any], results_folder: Path):
        """
        Initialize the ResultsSavingManager.

        Args:
            results_config (Dict[str, Any]): The configuration for saving results.
            base_folder (Path): The base directory where results should be saved.
        """
        self.results_config = results_config
        self.results_folder = results_folder

    def save_result(self,  result_config, data: Any):
        """
        Save a specific result according to the configuration.

        Args:
            result_name (str): The key name of the result in the configuration.
            data (Any): The data to save (could be a DataFrame, dict, etc.).
        """

        # Construct the save path
        save_dir = self.results_folder / self.results_config.dataset_name / self.results_config.model_name / result_config.get('folder')
        save_dir.mkdir(parents=True, exist_ok=True)
        file_path = save_dir / result_config.get('filename')

        # Save the data in the specified formats
        format = result_config.get('format')
        if isinstance(format, list):
            for fmt in format:
                self._save_in_format(data, file_path, fmt)
        else:
            self._save_in_format(data, file_path, format)


    def _save_in_format(self, data: Any, file_path: Path, fmt: str):
        """
        Save data in the specified format.

        Args:
            data (Any): The data to save.
            file_path (Path): The base path for the file.
            fmt (str): The format in which to save the data.
        """
        if fmt == 'json':
            if isinstance(data, pd.DataFrame):
              data.to_json(file_path.with_suffix('.jsonl.gz'), lines=True, orient="records")
            elif isinstance(data, dict) or isinstance(data, list):
                with open(file_path.with_suffix('.json'), 'w') as f:
                    json.dump(data, f, indent=4)
            elif isinstance(data, (go.Figure, px.Figure)):
                data.write_json(file_path.with_suffix('.json'))
            else:
                raise ValueError("Unsupported data type for JSON format.")
        elif fmt == 'vaex':
          if isinstance(data, pd.DataFrame):
              # Convert pandas DataFrame to vaex DataFrame
              data = vaex.from_pandas(data)
              data.export_hdf5(file_path.with_suffix('.hdf5'))
          elif isinstance(data, vaex.dataframe.DataFrame):
              data.export_hdf5(file_path.with_suffix('.hdf5'))
          else:
              raise ValueError("Vaex format is supported only for pandas or vaex DataFrames.")

    def save(self, data):
        """
        Save multiple results based on the configuration.

        Args:
            results (Dict[str, Any]): A dictionary of results where keys correspond to configuration keys.
        """
        self.save_result(self.results_config.analysis_data, data)

In [226]:
saver = ResultsSavingPipline(experiment_config.results_manager, experiment_config.extraction_base)

In [227]:
saver.save(analysis_extraction_pipeline.analysis_data)

analysis


TypeError: Cannot export column of type: object (column word_pieces)

In [234]:
test = analysis_extraction_pipeline.analysis_data.copy()
test['word_pieces'] = test['word_pieces'].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else str(x))

data = vaex.from_pandas(test)
              # data.export_hdf5(file_path.with_suffix('.hdf5'))

In [236]:
data

#,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,x,y,labels,losses,token_ids,global_id,true_token_score,pred_token_score,k=3,boundary_clusters,k=4,entity_clusters,k=9,token_clusters,consistency_count,inconsistency_count,total_train_occurrences,local_token_entropy,token_max_entropy,dataset_token_entropy,local_word_entropy,word_max_entropy,dataset_word_entropy,tokenization_rate,tr_entity,pr_entity,error_type,O,B-PERS,I-PERS,B-ORG,I-ORG,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
0,0,0,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS]@#0@#0,[CLS],True,1.9899436,14.148881,-100,0.0,2,2_0_0_-100,nan,nan,--,--,--,--,--,--,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,[CLS],[CLS],Correct,0.9998694363654025,8.769738394191879e-06,6.694727972179793e-06,1.9705650503524905e-05,4.391843594402248e-06,4.165958257316903e-05,2.961267634728435e-06,3.930773418145427e-05,7.07308974377591e-06,0.002193777103510289,3.1699250014423126,0.9998694363654025,0.3142235196189175,16.240751,8.657773
1,0,1,الصالحية,الصالحية,الصالحية,الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,1.4812794,-11.263502,5,0.30585336685180664,40508,40508_0_1_5,0.14190297,0.14175494,cluster-1,B,cluster-0,LOC,cluster-4,B-LOC,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,LOC,LOC,Correct,0.2385060659278503,0.002742406276006275,0.00024047881771706227,0.017964338155514674,0.0008456161585208715,0.7364946364876386,0.0011613461750631232,0.0018911154926333441,0.0001539965090558382,0.9876010206513879,3.1699250014423126,0.7364946364876386,0.2329841057621969,12.008531,3.4977665
2,0,2,المفرق,المفرق,المفرق,المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,1.3047713,-11.326236,5,0.02198176272213459,32479,32479_0_2_5,0.3451689,0.359852,cluster-1,B,cluster-0,LOC,cluster-4,B-LOC,0,1,1,0.0,0.0,0.00013549617986548032,0.0,0.0,0.0001353515157905157,1,LOC,LOC,Correct,0.006638029728442043,0.0007060816414328372,0.00017631563410367505,0.005458890789316556,0.0006610226502781095,0.978258058844395,0.00775042507883928,0.0001659343578649882,0.00018524127532759308,0.19537993681111013,3.1699250014423126,0.978258058844395,0.3065964917964641,12.091125,1.3605253
3,0,3,-,-,-,-,O,-@#3@#0,O,True,-0.21807238,8.648234,0,0.001036583329550922,19,19_0_3_0,0.49308965,0.53374964,cluster-0,O,cluster-1,O,cluster-0,O,519,0,519,0.0,0.0,0.032859989836357496,0.0,0.0,0.02677608454232953,1,O,O,Correct,0.9989639705004354,8.774119059338832e-05,4.635531248827807e-05,0.00010090480921828606,0.00021995944138138995,0.00032090777933990955,5.764135914492221e-05,0.00012548204530412452,7.703756209412536e-05,0.014570774416938062,3.1699250014423126,0.9989639705004354,0.3139033997980572,8.703134,5.905618
4,0,4,غيث,غيث,غيث,غيث,B-PERS,غيث@#4@#0,B-PERS,True,10.017178,-4.6364408,1,0.022747064009308815,33014,33014_0_4_1,0.44131896,0.49781975,cluster-2,B,cluster-3,PERS,cluster-2,B-PERS,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,PERS,PERS,Correct,0.015198784322625397,0.9775096610686408,0.0029923838331696556,0.0015216177444385111,0.0008214130222628167,0.0005373563606751728,0.000331366108124931,0.0008729269539392964,0.00021449058612350527,0.1927811264969286,3.1699250014423126,0.9775096610686408,0.3063509571245691,12.379341,1.6540067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"29,706",960,10,للوليد,##ليد,"للو,##ليد",IGNORED,IGNORED,IGNORED@#10@#960,IGNORED,True,10.137572,-4.1560864,-100,0.0,10486,10486_960_10_-100,nan,nan,--,--,--,--,--,--,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,IGNORED,IGNORED,Correct,0.0012193179259654887,0.994973966829704,0.0005374711041256044,0.0024079626497240003,0.00019229777527696737,0.00028720317326205735,0.00013359255623841064,0.00020434514860932071,4.384283709426086e-05,0.05642913773193988,3.1699250014423126,0.994973966829704,0.31249352676734543,6.6763244,-5.634513
"29,707",960,11,بن,بن,بن,بن,I

In [237]:
pandas_df = data.to_pandas_df(['x', 'y', 'true_labels'])  # Select the required columns
px.scatter(pandas_df, x='x', y='y', color='true_labels')

# Pretraind and Train split

In [11]:
from transformers import AutoModel
pretrained_model = AutoModel.from_pretrained(config_manager.model_path)

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from experiment_utils.model_outputs import ModelOutputWorkflowManager, PretrainedModelOutputWorkflowManager
from experiment_utils.tokenization import TokenizationWorkflowManager
from experiment_utils.analysis import DataExtractor, UMAPTransformer, UtilityFunctions, LabelAligner, ClusterAnalysis, DataAnnotator

In [17]:
model_outputs = ModelOutputWorkflowManager(model, data_manager, config_manager.training_config)

INFO:root:Training Config validated successfully
INFO:root:Processing train Split
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02, lower_case: False
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/260 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:root:Processing test Split
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02, lower_case: False


  0%|          | 0/121 [00:00<?, ?it/s]

In [18]:
pre_model_outputs = PretrainedModelOutputWorkflowManager(pretrained_model, data_manager, config_manager.training_config, "test")

INFO:root:Training Config validated successfully
INFO:root:Loading Preprocessor: aubmindlab/bert-base-arabertv02
INFO:root:Loading Tokenizer: aubmindlab/bert-base-arabertv02, lower_case: False


  0%|          | 0/121 [00:00<?, ?it/s]

In [19]:
tokenization_outputs = TokenizationWorkflowManager(data_manager.corpus, config_manager.tokenization_config)

INFO:root:Tokenization Config validated successfully
INFO:root:Loading Tokenizer aubmindlab/bert-base-arabertv02
INFO:root:Loading Preprocessor aubmindlab/bert-base-arabertv02
INFO:root:Processing train split


  0%|          | 0/4149 [00:00<?, ?it/s]

INFO:root:Extracting train subwords


  0%|          | 0/4149 [00:00<?, ?it/s]

INFO:root:Processing test split


  0%|          | 0/961 [00:00<?, ?it/s]

In [22]:
transformer = UMAPTransformer(config_manager.umap_config)
aligner = LabelAligner(
            evaluation_results.entity_outputs['y_pred'].copy(), tokenization_outputs.test
        )

INFO:root:UMAP Config validated successfully


In [23]:
extractor = DataExtractor(
                        model_outputs=model_outputs.get_split('test'),
                        tokenization_outputs=tokenization_outputs.get_split('test'),
                        aligner=aligner,
                        transformer=transformer
                        )

INFO:root:Extracting model features...
INFO:root:Extracting tokenization features...
INFO:root:Aligning labels...


NameError: name 'aligner' is not defined

In [ ]:
pre_extractor = DataExtractor(
                        model_outputs=pre_model_outputs.get_split('test'),
                        transformer=transformer
                        )

In [ ]:
analysis_df = extractor.to_df()
pretrained_coordinates = pre_extractor.to_df({'x': 'pre_x', 'y': 'pre_y'}, ['global_id', 'pre_x', 'pre_y'])


In [ ]:
import logging
from scipy.spatial.distance import cdist

from sklearn.metrics import silhouette_samples, homogeneity_completeness_v_measure
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize


In [ ]:
clustering_analyser = ClusterAnalysis(extractor, analysis_df, config_manager.clustering_config)
average_silhouette_score, kmeans_metrics, clustering_df, centroid_avg_similarity = clustering_analyser.calculate()
merged_data = analysis_df.merge(clustering_df, on='global_id', how='left')

In [ ]:
merged_data

In [ ]:
from experiment_utils.analysis import TokenConsistencyCalculator, TokenEntropyCalculator, WordEntropyCalculator, PredictionEntropyCalculator



In [ ]:
data_annotator = DataAnnotator(
                tokenization_outputs.train_subwords,
                merged_data,
                pretrained_coordinates,
                data_manager.data.get('train'),
                model_outputs.test,
                data_manager.corpus['labels_map'],
            )

In [ ]:
pretrained_coordinates

In [ ]:
data_annotator.annotate_all()

In [205]:
analysis_manager = AnalysisWorkflowManager(
            config_manager, evaluation_results,
            tokenization_outputs, model_outputs,
            pre_model_outputs, data_manager, "test"
        )

INFO:root:UMAP Config validated successfully


In [206]:
analysis_data, average_silhouette_score, kmeans_metrics, centroids_avg_similarities = analysis_manager.run()

INFO:root:Extracting model features...
INFO:root:Extracting tokenization features...
INFO:root:Aligning labels...
INFO:root:Applying UMAP...
INFO:root:Applying UMAP...
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


UMAP(angular_rp_forest=True, metric='cosine', n_jobs=1, random_state=1, verbose=True)
Thu Aug  8 14:06:31 2024 Construct fuzzy simplicial set
Thu Aug  8 14:06:31 2024 Finding Nearest Neighbors
Thu Aug  8 14:06:31 2024 Building RP forest with 14 trees
Thu Aug  8 14:06:33 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	 6  /  15
	Stopping threshold met -- exiting after 6 iterations
Thu Aug  8 14:06:38 2024 Finished Nearest Neighbor Search
Thu Aug  8 14:06:38 2024 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Thu Aug  8 14:07:08 2024 Finished embedding


INFO:root:Extracting model features...
INFO:root:Extracting tokenization features...
INFO:root:Applying UMAP...
INFO:root:Applying UMAP...
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


...

INFO:root:Clustering Config validated successfully
INFO:root:Calculating Silhouette Score
INFO:root:Processing K=3
INFO:root:Processing K=4
INFO:root:Processing K=9
INFO:root:Calculating Centorid Average Similarity Matrix
INFO:root:Annotating all...
INFO:root:Annotating consistency...


	completed  180  /  200 epochs
Thu Aug  8 14:07:34 2024 Finished embedding


Extract Subwords:   0%|          | 0/23049 [00:00<?, ?it/s]

Calculate Consistency:   0%|          | 0/29711 [00:00<?, ?it/s]

INFO:root:Annotating token entropy...
INFO:root:Annotating word entropy...
INFO:root:Annotating entity...
INFO:root:Annotating error types...
INFO:root:Annotating prediction entropy...
INFO:root:Annotating pretrained coordinates...
INFO:root:Analysis workflow execution time: 110.20017743110657 seconds


In [152]:
tr_df = analysis_manager.generate_train_df()

4

In [207]:
analysis_data

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
0,0,0,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS]@#0@#0,[CLS],True,...,0.000042,2.961268e-06,0.000039,0.000007,0.002194,3.169925,0.999869,0.314224,16.240751,8.657773
1,0,1,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,...,0.736495,1.161346e-03,0.001891,0.000154,0.987601,3.169925,0.736495,0.232984,12.008531,3.497766
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,0.978258,7.750425e-03,0.000166,0.000185,0.195380,3.169925,0.978258,0.306596,12.091125,1.360525
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,0.000321,5.764136e-05,0.000125,0.000077,0.014571,3.169925,0.998964,0.313903,8.703134,5.905618
4,0,4,غيث,غيث,[غيث],غيث,B-PERS,غيث@#4@#0,B-PERS,True,...,0.000537,3.313661e-04,0.000873,0.000214,0.192781,3.169925,0.977510,0.306351,12.379341,1.654007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29706,960,10,للوليد,##ليد,"[للو, ##ليد]",IGNORED,IGNORED,IGNORED@#10@#960,IGNORED,True,...,0.000287,1.335926e-04,0.000204,0.000044,0.056429,3.169925,0.994974,0.312494,6.676324,-5.634513
29707,960,11,بن,بن,[بن],بن,I-PERS,بن@#11@#960,I-PERS,True,...,0.000063,1.495592e-04,0.000086,0.000138,0.063753,3.169925,0.994191,0.312217,6.765226,-5.459598
29708,960,12,طلال,طلال,[طلال],طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,0.000067,1.289662e-04,0.000155,0.000110,0.038872,3.169925,0.996783,0.313133,6.672010,-5.624885
29709,960,13,.,.,[.],.,O,.@#13@#960,O,True,...,0.000005,6.299709e-07,0.000013,0.000004,0.000704,3.169925,0.999963,0.314256,10.402867,9.328933


In [220]:
analysis_data

,sentence_ids,token_positions,words,tokens,word_pieces,core_tokens,true_labels,token_selector_id,pred_labels,agreements,...,B-LOC,I-LOC,B-MISC,I-MISC,prediction_entropy,prediction_max_entropy,confidence,variability,pre_x,pre_y
0,0,0,[CLS],[CLS],[CLS],[CLS],[CLS],[CLS]@#0@#0,[CLS],True,...,0.000042,2.961268e-06,0.000039,0.000007,0.002194,3.169925,0.999869,0.314224,16.240751,8.657773
1,0,1,الصالحية,الصالحية,[الصالحية],الصالحية,B-LOC,الصالحية@#1@#0,B-LOC,True,...,0.736495,1.161346e-03,0.001891,0.000154,0.987601,3.169925,0.736495,0.232984,12.008531,3.497766
2,0,2,المفرق,المفرق,[المفرق],المفرق,B-LOC,المفرق@#2@#0,B-LOC,True,...,0.978258,7.750425e-03,0.000166,0.000185,0.195380,3.169925,0.978258,0.306596,12.091125,1.360525
3,0,3,-,-,[-],-,O,-@#3@#0,O,True,...,0.000321,5.764136e-05,0.000125,0.000077,0.014571,3.169925,0.998964,0.313903,8.703134,5.905618
4,0,4,غيث,غيث,[غيث],غيث,B-PERS,غيث@#4@#0,B-PERS,True,...,0.000537,3.313661e-04,0.000873,0.000214,0.192781,3.169925,0.977510,0.306351,12.379341,1.654007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29706,960,10,للوليد,##ليد,"[للو, ##ليد]",IGNORED,IGNORED,IGNORED@#10@#960,IGNORED,True,...,0.000287,1.335926e-04,0.000204,0.000044,0.056429,3.169925,0.994974,0.312494,6.676324,-5.634513
29707,960,11,بن,بن,[بن],بن,I-PERS,بن@#11@#960,I-PERS,True,...,0.000063,1.495592e-04,0.000086,0.000138,0.063753,3.169925,0.994191,0.312217,6.765226,-5.459598
29708,960,12,طلال,طلال,[طلال],طلال,I-PERS,طلال@#12@#960,I-PERS,True,...,0.000067,1.289662e-04,0.000155,0.000110,0.038872,3.169925,0.996783,0.313133,6.672010,-5.624885
29709,960,13,.,.,[.],.,O,.@#13@#960,O,True,...,0.000005,6.299709e-07,0.000013,0.000004,0.000704,3.169925,0.999963,0.314256,10.402867,9.328933


In [219]:
old_df


,global_id,token_id,word_id,sen_id,token_ids,label_ids,first_tokens_freq,first_tokens_consistency,first_tokens_inconsistency,words,...,I-PERS,B-ORG,I-ORG,B-LOC,I-LOC,B-MISC,I-MISC,3_clusters,4_clusters,9_clusters
0,0,2,0,0,[CLS]@#0@#0,-100,0,0,0,[CLS],...,0.000007,0.000020,0.000004,0.000042,2.961300e-06,0.000039,0.000007,IGNORED,IGNORED,IGNORED
1,1,40508,1,0,الصالحية@#0@#1,5,0,0,0,الصالحية,...,0.000240,0.017964,0.000846,0.736495,1.161344e-03,0.001891,0.000154,cluster-2,cluster-2,cluster-2
2,2,32479,2,0,المفرق@#0@#2,5,1,0,1,المفرق,...,0.000176,0.005459,0.000661,0.978258,7.750400e-03,0.000166,0.000185,cluster-2,cluster-2,cluster-2
3,3,19,3,0,-@#0@#3,0,409,409,0,-,...,0.000046,0.000101,0.000220,0.000321,5.764150e-05,0.000125,0.000077,cluster-1,cluster-1,cluster-7
4,4,33014,4,0,غيث@#0@#4,1,0,0,0,غيث,...,0.002992,0.001522,0.000821,0.000537,3.313662e-04,0.000873,0.000214,cluster-0,cluster-0,cluster-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29706,29706,10486,10,960,IGNORED@#960@#10,-100,0,0,0,للوليد,...,0.000537,0.002408,0.000192,0.000287,1.335925e-04,0.000204,0.000044,IGNORED,IGNORED,IGNORED
29707,29707,463,11,960,بن@#960@#11,2,96,86,10,بن,...,0.994191,0.000158,0.001441,0.000063,1.495592e-04,0.000086,0.000138,cluster-0,cluster-0,cluster-5
29708,29708,7031,12,960,طلال@#960@#12,2,2,1,1,طلال,...,0.996783,0.000096,0.000972,0.000067,1.289662e-04,0.000155,0.000110,cluster-0,cluster-0,cluster-5
29709,29709,20,13,960,.@#960@#13,0,3402,3402,0,.,...,0.000002,0.000004,0.000003,0.000005,6.300000e-07,0.000013,0.000004,cluster-1,cluster-3,cluster-8


In [224]:
col_order = [
    'global_id', 'token_ids', 'token_positions',
    'sentence_ids', 'token_selector_id', 'labels',
    'consistency_count', 'inconsistency_count', 'total_train_occurrences',
    'words', 'word_pieces', 'tokens', 'core_tokens', 'true_labels',
    'pred_labels', 'agreements', 'losses', 'x', 'y', 'tokenization_rate',
    'local_token_entropy', 'local_word_entropy',
    'tr_entity', 'pr_entity', 'error_type',
    'prediction_entropy', 'confidence', 'variability', 'O', 'B-PERS', 'I-PERS', 'B-ORG', 'I-ORG', 'B-LOC',
    'I-LOC', 'B-MISC', 'I-MISC', 'k=3', 'k=4', 'k=9',
             ]
analysis_data[[c for c in analysis_data.columns if c not in col_order]]

,true_token_score,pred_token_score,boundary_clusters,entity_clusters,token_clusters,token_max_entropy,dataset_token_entropy,word_max_entropy,dataset_word_entropy,prediction_max_entropy,pre_x,pre_y
0,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.0,-1.000000,3.169925,16.240751,8.657773
1,0.141903,0.141755,B,LOC,B-LOC,-1.000000,-1.000000,-1.0,-1.000000,3.169925,12.008531,3.497766
2,0.345169,0.359852,B,LOC,B-LOC,0.000000,0.000135,0.0,0.000135,3.169925,12.091125,1.360525
3,0.493090,0.533750,O,O,O,0.000000,0.032860,0.0,0.026776,3.169925,8.703134,5.905618
4,0.441319,0.497820,B,PERS,B-PERS,-1.000000,-1.000000,-1.0,-1.000000,3.169925,12.379341,1.654007
...,...,...,...,...,...,...,...,...,...,...,...,...
29706,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.0,-1.000000,3.169925,6.676324,-5.634513
29707,0.583306,0.638236,I,PERS,I-PERS,2.321928,0.009743,2.0,0.009043,3.169925,6.765226,-5.459598
29708,0.574142,0.628047,I,PERS,I-PERS,1.000000,0.000271,1.0,0.000271,3.169925,6.672010,-5.624885
29709,0.672943,0.706441,O,O,O,0.000000,0.163114,0.0,0.162979,3.169925,10.402867,9.328933


In [ ]:
model_cols = []
tokenization_cols = []
transform_cols = []

annotation_cols = []
clustering_cols = []